In [1]:
pip install numpy pandas matplotlib seaborn scikit-learn ipython

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import seaborn as sns
import pandas as pd

import random

from datetime import datetime, timedelta

from matplotlib import rcParams
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from IPython.display import display

import sys

In [ ]:
# Cleaning utilities functions
import pandas as pd

def drop_invalid_dates(df, col):
    """
    Detects invalid dates (Na) in a column of a DataFrame.
    Drops invalid rows with invalid dates.
    
    :param df: DataFrame containing the dates
    :param col: Name of the column to check for invalid dates
    :return: DataFrame with valid dates and list of invalid dates
    """
    # Convert the column to datetime
    parsed_dates = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
    
    # Find invalid dates
    invalid_dates = df[col][parsed_dates.isna()]
    
    # Find rows with invalid dates (NaT)
    invalid_dates_mask = parsed_dates.isna()
    
    if invalid_dates.empty:
        print(f"No invalid dates found in column '{col}'.")
    else:
        print(f"Dropping invalid rows in column '{col}'...")

    # Drop rows with invalid dates
    df_cleaned = df[~invalid_dates_mask]
    
    return df_cleaned, invalid_dates


def drop_invalid_rows(df):
    """
    Drops rows with missing values (containing NaN values) 
    Drop rows with invalid time; START >= END.

    :param df: DataFrame to clean
    :return: Cleaned DataFrame
    """
    initial_len = len(df)

    # Drop rows where START == END
    if 'START' in df.columns and 'END' in df.columns:
        df = df[df['START'] < df['END']]

    # Drop rows with any NaN values
    df = df.dropna()

    print(f"Dropped {initial_len - len(df)} rows with missing values or START >= END.")
    return df

### Environmental Configurations

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data_path = os.path.join("..", "Data")
schedules_df = pd.read_excel(os.path.join(data_path, "Schedules.xlsx"))
data_df = pd.read_excel(os.path.join(data_path, "Data.xlsx"))
clients_df = pd.read_excel(os.path.join(data_path, "Clients.xlsx"))
transl_cost_pairs_df = pd.read_excel(os.path.join(data_path, "TranslatorsCost+Pairs.xlsx"))

In [4]:
data_df['START'] = pd.to_datetime(data_df['START'], errors='coerce')

### Envirinmental Variables

In [5]:
RANDOM_SEED = 42

TRANSLATORS_UNAVAILABLE = []

wildcards = [None, "Quality", "Time", "Cost"]
task_types = data_df["TASK_TYPE"].unique()
unique_language_pairs = data_df[["SOURCE_LANG", "TARGET_LANG"]].drop_duplicates().reset_index(drop=True)

> **Note:** `TRANSLATORS_UNAVAILABLE` is a list for keeping track of translators alredy assigned or performing a task

In [6]:
## TODO we have to think what to do with translators that can perform multiple tasks

### Data Cleaning

In [7]:
# Function to detect invalid rows
data_df, start_invalid_dates = drop_invalid_dates(data_df, 'START')
data_df, end_invalid_dates = drop_invalid_dates(data_df, 'END')
data_df, delivered_invalid_dates = drop_invalid_dates(data_df, 'DELIVERED')
data_df, delivered_invalid_dates = drop_invalid_dates(data_df, 'ASSIGNED')

print(" ")

# Show the invalid dates in each column
print("Invalid START dates:\n", start_invalid_dates, "\n")
print("Invalid END dates:\n", end_invalid_dates, "\n")
print("Invalid DELIVERED dates:\n", delivered_invalid_dates, "\n")

Dropping invalid rows in column 'START'...
No invalid dates found in column 'END'.
No invalid dates found in column 'DELIVERED'.
No invalid dates found in column 'ASSIGNED'.
 
Invalid START dates:
 91127   NaT
Name: START, dtype: datetime64[ns] 

Invalid END dates:
 Series([], Name: END, dtype: datetime64[ns]) 

Invalid DELIVERED dates:
 Series([], Name: ASSIGNED, dtype: datetime64[ns]) 



In [8]:
data_df = drop_invalid_rows(data_df)

Dropped 1853 rows with missing values or START == END.


#### Task Information
##### General Info
- **PROJECT_ID**: Project code (additional info, likely not necessary).
- **PM**: Responsible management team.
- **TASK_ID**: Task code.
##### Dates
- **START**: Task start date.
- **END**: Theoretical task delivery date (can be compared with `DELIVERED` to check for delays).
##### Task Type (`TASK_TYPE`)
Some considerations must be taken into account:
- **DTP**: Desktop-Publishing tasks.
- **Engineering**: Engineering tasks such as file conversions, coding, etc.
- **LanguageLead**: Linguistic management tasks. Assigned to highly experienced and quality-oriented individuals who regularly work on the project.
- **Management**: General management tasks.
- **Miscellaneous**: Various linguistic tasks.
- **PostEditing**: Post-editing tasks. Similar to Translation tasks but with slightly different skills required for the TRANSLATOR.
- **ProofReading**: Full review of a Translation or PostEditing. Always follows a Translation or PostEditing. The TRANSLATOR assigned must have more 
experience than the person who performed the initial step.
- **Spotcheck**: Partial review of a Translation or PostEditing. Similar conditions as ProofReading.
- **TEST**: Test required to qualify for working with a client. Should be assigned to the most experienced and high-quality TRANSLATOR 
for the client or topic, regardless of price but considering the deadline.
- **Training**: Translator experience and quality are not considered.
- **Translation**: Translation task. The translator’s quality can be slightly lower if the ProofReading (not Spotcheck) is done by a superior. If 
Spotcheck is done, the required quality must be met.
##### Languages
- **SOURCE_LANG**: Source language.
- **TARGET_LANG**: Target language.
##### Workflow 
- **TRANSLATOR**: Translator responsible for the task.
- **ASSIGNED**: Time when the task is assigned (pre-notice) to the TRANSLATOR.
- **READY**: Time when the TRANSLATOR is notified they can start.
- **WORKING**: Time when the TRANSLATOR starts the task.
- **DELIVERED**: Time when the TRANSLATOR delivers the task.
- **RECEIVED**: Time when the PM receives the task.
- **CLOSE**: Time when the PM marks the task as completed.
##### Cost & Quality
- **FORECAST**: Estimated hours for completion.
- **HOURLY_RATE**: Task hourly rate.
- **COST**: Total task cost.
- **QUALITY_EVALUATION**: Quality control evaluation.
##### Client Info
- **MANUFACTURER**: Client.
- **MANUFACTURER_SECTOR**: Level 1 client categorization.
- **MANUFACTURER_INDUSTRY_GROUP**: Level 2 client categorization.
- **MANUFACTURER_INDUSTRY**: Level 3 client categorization.
- **MANUFACTURER_SUBINDUSTRY**: Level 4 client categorization.

In [9]:
class Task:
    def __init__(self, **kwargs):
        """
        A class used to represent a Task. It initializes the attributes dynamically 
        using the keyword arguments passed. Default values are provided for certain fields.
        """
        self.PROJECT_ID = kwargs.get('PROJECT_ID', None)
        self.TASK_ID = kwargs.get('TASK_ID', None)
        self.ASSIGNED = kwargs.get('ASSIGNED', None)
        self.END = kwargs.get('END', None)
        self.SELLING_HOURLY_PRICE = kwargs.get('SELLING_HOURLY_PRICE', None)
        self.MIN_QUALITY = kwargs.get('MIN_QUALITY', None)
        self.WILDCARD = kwargs.get('WILDCARD', None) 
        self.TASK_TYPE = kwargs.get('TASK_TYPE', None)
        self.SOURCE_LANG = kwargs.get('SOURCE_LANG', None)
        self.TARGET_LANG = kwargs.get('TARGET_LANG', None)
        self.MANUFACTURER = kwargs.get('MANUFACTURER', None)
        self.MANUFACTURER_SECTOR = kwargs.get('MANUFACTURER_SECTOR', None)
        self.MANUFACTURER_INDUSTRY_GROUP = kwargs.get('MANUFACTURER_INDUSTRY_GROUP', None)
        self.MANUFACTURER_INDUSTRY = kwargs.get('MANUFACTURER_INDUSTRY', None)
        self.MANUFACTURER_SUBINDUSTRY = kwargs.get('MANUFACTURER_SUBINDUSTRY', None)
        
        # Optional attributes with None default value
        self.START = kwargs.get('START', None)
        self.PM = kwargs.get('PM', None)
        self.TRANSLATOR = kwargs.get('TRANSLATOR', None)
        self.READY = kwargs.get('READY', None)
        self.WORKING = kwargs.get('WORKING', None)
        self.DELIVERED = kwargs.get('DELIVERED', None)
        self.RECEIVED = kwargs.get('RECEIVED', None)
        self.CLOSE = kwargs.get('CLOSE', None)
        self.FORECAST = kwargs.get('FORECAST', None)
        self.HOURLY_RATE = kwargs.get('HOURLY_RATE', None)
        self.COST = kwargs.get('COST', None)
        self.QUALITY_EVALUATION = kwargs.get('QUALITY_EVALUATION', None)
        
    
    def __str__(self):
        return (
            f"Task Details:\n"
            f"  - Task ID: {self.TASK_ID}\n"
            f"  - Type: {self.TASK_TYPE}\n"
            f"  - Client: {self.MANUFACTURER}\n"
            f"  - Sector: {self.MANUFACTURER_SECTOR}\n"
            f"  - Industry (Subsector): {self.MANUFACTURER_INDUSTRY}\n"
            f"  - Start: {self.START}\n"
            f"  - Budget: {self.SELLING_HOURLY_PRICE}\n"
            f"  - Quality: {self.MIN_QUALITY}\n"
            f"  - Wildcard: {self.WILDCARD}\n"
            f"  - Source Language: {self.SOURCE_LANG}\n"
            f"  - Target Language: {self.TARGET_LANG}"
        )

### Split Data

In [10]:
# Split into train and validation (e.g., 80% train, 20% validation)
train_df, validation_df = train_test_split(data_df, test_size=0.2, random_state=42)

In [11]:
def drop_and_save_translator_labels(df, translator_column="TRANSLATOR"):
    """
    Extracts and removes translators from test_df, then saves them as a label dict.
    
    Args:
        df (pd.DataFrame): The dataframe containing translator data.
        translator_column (str): The column name that holds the translator labels.
        
    Returns:
        pd.DataFrame: The test_df without the translators column.
        dict: Dictionary of translator labels {index: translators}
    """
    if translator_column not in df.columns:
        raise ValueError(f"Column '{translator_column}' not found in test_df.")
    
    # Extract labels
    translator_labels = df[translator_column].to_dict()
    
    # Drop the column from the DataFrame
    df = df.drop(columns=[translator_column])
    
    return df, translator_labels

train_df_clean, train_translator_labels = drop_and_save_translator_labels(train_df)
validation_df_clean, validation_translator_labels = drop_and_save_translator_labels(validation_df)

In [12]:
train_df_clean.head()

,PROJECT_ID,PM,TASK_ID,START,END,TASK_TYPE,SOURCE_LANG,TARGET_LANG,ASSIGNED,READY,WORKING,DELIVERED,RECEIVED,CLOSE,FORECAST,HOURLY_RATE,COST,QUALITY_EVALUATION,MANUFACTURER,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY
68024,211978,RMT,10236246,2013-08-02 11:30:00,2013-08-05 18:00:00,Translation,English,Spanish (Iberian),2013-08-02 12:20:48,2013-08-02 12:21:05,2013-08-02 17:59:27,2013-08-02 20:14:33,2013-08-05 08:22:52,2013-08-05 08:22:54,1.34,17,22.78,9,TrueConnect,Communication Services,Interactive Media & Services,Internet Services & Infrastructure,Internet Services & Infrastructure
420702,219630,RMT,10975047,2021-03-01 12:55:00,2021-03-01 12:52:00,ProofReading,English,Spanish (Iberian),2021-03-01 09:05:53,2021-03-01 11:48:23,2021-03-01 11:59:28,2021-03-01 12:11:28,2021-03-01 12:34:20,2021-03-01 12:39:56,0.02,19,0.38,6,FrontierTech,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals"
522938,220738,KMT,11172175,2022-07-18 10:20:00,2022-07-19 16:00:00,Miscellaneous,English,Spanish (Iberian),2022-07-18 10:43:25,2022-07-18 10:43:28,2022-07-18 12:46:01,2022-07-18 16:04:25,2022-07-18 17:05:46,2022-07-18 17:05:57,2.50,18,45.00,5,NanoWorks Research Corp,Health Care,Life Sciences Tools and Services,Life Sciences Tools and Services,Life Sciences Tools and Services
496960,220738,KMT,11115192,2022-03-10 22:27:00,2022-03-11 09:00:00,Miscellaneous,English,Spanish (LA),2022-03-10 22:27:11,2022-03-10 22:28:23,2022-03-10 22:31:07,2022-03-10 22:35:06,2022-03-11 01:08:58,2022-03-11 01:09:05,0.03,14,0.42,6,NanoWorks Research Corp,Health Care,Life Sciences Tools and Services,Life Sciences Tools and Services,Life Sciences Tools and Services
449116,219635,RMT,11024780,2021-07-15 13:00:00,2021-07-15 20:59:00,ProofReading,English,Galician,2021-07-12 10:14:03,2021-07-15 14:03:50,2021-07-15 15:00:22,2021-07-15 16:22:14,2021-07-15 16:25:02,2021-07-15 16:25:02,0.05,12,0.60,7,TrueConnect,Communication Services,Interactive Media & Services,Internet Services & Infrastructure,Internet Services & Infrastructure


In [13]:
for i, (key, value) in enumerate(validation_translator_labels.items()):
    print(f"Task Index: {key}, Translator: {value}")
    if i == 10:
        break

Task Index: 159998, Translator: Erico
Task Index: 223845, Translator: Francisco Jose
Task Index: 152923, Translator: Almudena Fiamma
Task Index: 245250, Translator: Lionel
Task Index: 374407, Translator: Priscila
Task Index: 93960, Translator: Christopher
Task Index: 416650, Translator: Eliseo
Task Index: 302712, Translator: Casiano
Task Index: 424880, Translator: Edmundo
Task Index: 508616, Translator: Columbano Cleofas
Task Index: 233081, Translator: Fortunato


### Explanation of what is happening

In the global scope:

0. We use as base of useful features the dataframe `transl_cost_pairs_df`
It includes:
- TRANSLATOR: Translator name.
- SOURCE_LANG: Source language.
- TARGET_LANG: Target language.
- HOURLY_RATE: Cost per hour.


1. We compute the **average proportional delay** (speed) from the `data_df` for each translator and merge it with this base dataframe


Now for the each task:

2. We start to the strict filter:
    - Filter of **languages**: only consider the translators who offer this translation
    - Filter of **price**: only consider the prices below the threshold
    - Filter of **quality** by language: this is done by making an average of the quality of these languages for each translator, and then using it as a threshold. 
    - Filter of **availability**: if the taks lasts less than 7 days we check whether the translator will even work before the theoretical deadline (it is 7 days because everyone works at least once a week)

3. We do a **weighted knn**:
    - We do it on the *perfect point* (price = 0, quality = 10, speed = 100%, experience = 10... (orientative values))
    - The weights are chosen by the wildcards and by the expereince required for the type of task. A weighted knn, after normalizing, it distorts the chosen axis size to give more or less weight. 
    - The similarity score is the final ranking. 

4. Outcome possibilities: 
    - We get None or too few translators: if it is None we use the wildcard to completely ignore that factor in the strict filter
    - We get a lot of translators: (we need the rest of features to give better recommendations)

5. Calculate accuracy and find a way to retrieve explainations
    

In [24]:
# ----- THESE ARE GENERAL FUNCTIONS THAT CAN BE USED TO ACTUALIZE THE DATA IF NEW TASKS WERE ADDED TO THE DATASET -----
def compute_delay_percentage(data_df, transl_cost_pairs_df):
    """
    Compute the delay percentage of each translator based on task completion times.
    Negative values indicate early delivery, positive values indicate late delivery.
    
    Args:
        data_df (pd.DataFrame): Task data with 'TRANSLATOR', 'START', 'END', 'DELIVERED'
        transl_cost_pairs_df (pd.DataFrame): Translator costs with 'TRANSLATOR', 'COST'
    
    Returns:
        pd.DataFrame: Merged dataframe with 'TRANSLATOR', 'COST', 'AVG_DELAY_PERCENTAGE'
    """
    # Convert date columns to datetime
    date_cols = ['START', 'END', 'DELIVERED']
    for col in date_cols:
        data_df[col] = pd.to_datetime(data_df[col], errors='coerce')

    # Calculate duration and delay percentage
    duration = data_df['END'] - data_df['START']
    duration = duration.replace(pd.Timedelta(0), pd.NaT)  # Prevent division by zero
    data_df['DELAY_PERCENTAGE'] = (
        (data_df['DELIVERED'] - data_df['END']) / duration * 100
    )

    # Clean invalid values and cap at 100%
    data_df['DELAY_PERCENTAGE'] = (
        data_df['DELAY_PERCENTAGE']
        .replace([np.inf, -np.inf, np.nan], 0)
        .clip(upper=100, lower=-100)
    )

    # Calculate average delay with proper rounding
    avg_delay = (
        data_df.groupby('TRANSLATOR')['DELAY_PERCENTAGE']
        .mean()
        .round(2)
        .reset_index()
        .rename(columns={'DELAY_PERCENTAGE': 'AVG_DELAY_PERCENTAGE'})
    )

    # Merge with cost data and fill missing values
    return (
        transl_cost_pairs_df.merge(avg_delay, on='TRANSLATOR', how='left')
        .fillna({'AVG_DELAY_PERCENTAGE': 0})
    )


def compute_number_tasks(data_df, translators_attributes_df):
    """
    Computes the number of tasks for each translator.
    
    Args:
        data_df (pd.DataFrame): 
            DataFrame containing the data of the tasks.
        df_filtered (pd.DataFrame): 
            DataFrame containing the filtered translators' attributes.

    Returns:
        translators_attributes_df (pd.DataFrame) with the delay_percentage.
    """
    #TAKE INTO ACCOUNT: Task count is added to the dataset to help judge reliability (e.g if the quality of translator is calculated using a few tasks). 

    # Count the number of tasks each translator has done
    task_counts = data_df.groupby('TRANSLATOR').size().reset_index(name='NUM_TASKS')

    # Merge the task counts into the filtered dataframe
    translators_attributes_df = translators_attributes_df.merge(task_counts, on='TRANSLATOR', how='left')

    # Fill missing values (i.e., translators with no tasks) with 0
    translators_attributes_df['NUM_TASKS'] = translators_attributes_df['NUM_TASKS'].fillna(0).astype(int)

    return translators_attributes_df

In [ ]:

#---- MULTITASKING-AWARE AVAILABILITY FUNCTION -----
# Below we preprocess and detect multitaskers + compatible task-type pairs

# Ensure datetime fields are parsed
train_df['WORKING'] = pd.to_datetime(train_df['WORKING'], errors='coerce')
train_df['DELIVERED'] = pd.to_datetime(train_df['DELIVERED'], errors='coerce')

# Filter for valid entries
task_df = train_df.dropna(subset=['TRANSLATOR', 'WORKING', 'DELIVERED']).copy()

# Detect multitasking translators
def detect_multitasking(group):
    group = group.sort_values(by='WORKING')
    multitask_flags = []
    current_end = pd.Timestamp.min
    for _, row in group.iterrows():
        multitask_flags.append(row['WORKING'] < current_end)
        current_end = max(current_end, row['DELIVERED'])
    group['MULTITASKING'] = multitask_flags
    return group

task_df = task_df.groupby('TRANSLATOR').apply(detect_multitasking).reset_index(drop=True)
multitaskers_set = set(task_df[task_df['MULTITASKING']]['TRANSLATOR'].unique())

# Detect multitaskable task-type combinations
from itertools import combinations

def get_overlapping_type_pairs(group):
    group = group.sort_values(by='WORKING')
    pairs = set()
    for i in range(len(group)):
        for j in range(i+1, len(group)):
            task1 = group.iloc[i]
            task2 = group.iloc[j]
            if task1['DELIVERED'] > task2['WORKING'] and task2['DELIVERED'] > task1['WORKING']:
                type_pair = tuple(sorted([task1['TASK_TYPE'], task2['TASK_TYPE']]))
                pairs.add(type_pair)
    return pairs

all_task_type_pairs = set()
for translator, group in task_df.groupby('TRANSLATOR'):
    all_task_type_pairs.update(get_overlapping_type_pairs(group))

# Multitasking-aware availability function
def is_translator_available(translator, new_start, new_end, new_type, task_df, multitaskers_set, allowed_type_pairs):
    past_tasks = task_df[task_df['TRANSLATOR'] == translator]
    for _, task in past_tasks.iterrows():
        if new_start < task['DELIVERED'] and new_end > task['WORKING']:
            if translator not in multitaskers_set:
                return False  # No overlap allowed
            task_pair = tuple(sorted([new_type, task['TASK_TYPE']]))
            if task_pair not in allowed_type_pairs:
                return False  # Overlap allowed only for seen task-type combos
    return True


In [ ]:

def available_translators(task, translators_attributes_df, schedules_df, TRANSLATORS_UNAVAILABLE):
    """
    Checks if translators are available for the task based on their weekly working schedule
    and multitasking-aware logic.
    
    Args:
        task (Task object): Task to evaluate.
        translators_attributes_df (pd.DataFrame): Translator attributes.
        schedules_df (pd.DataFrame): Weekly schedules.
        TRANSLATORS_UNAVAILABLE (list): Translators known to be unavailable.
    
    Returns:
        pd.DataFrame: Filtered translators who are available.
    """
    # Step 1: Remove explicitly unavailable translators
    df_filtered = translators_attributes_df[~translators_attributes_df['TRANSLATOR'].isin(TRANSLATORS_UNAVAILABLE)].copy()

    # Step 2: Merge with schedules and check weekly availability
    task_day = task.ASSIGNED.strftime('%a').upper()
    df_filtered = df_filtered.merge(schedules_df, left_on='TRANSLATOR', right_on='NAME').drop(columns=['NAME'])

    def is_weekly_available(row):
        if row[task_day] != 1:
            return False
        task_start_time = timedelta(hours=task.ASSIGNED.hour, minutes=task.ASSIGNED.minute)
        task_end_time = task_start_time + timedelta(hours=1)  # Default duration
        work_start_time = timedelta(hours=row['START'].hour, minutes=row['START'].minute)
        work_end_time = timedelta(hours=row['END'].hour, minutes=row['END'].minute)
        return (task_start_time >= work_start_time) and (task_end_time < work_end_time)

    df_filtered['IS_WEEKLY_AVAILABLE'] = df_filtered.apply(is_weekly_available, axis=1)
    df_filtered = df_filtered[df_filtered['IS_WEEKLY_AVAILABLE']].drop(columns=['IS_WEEKLY_AVAILABLE'])

    # Step 3: Apply multitasking-aware availability
    def is_fully_available(row):
        return is_translator_available(
            row['TRANSLATOR'],
            task.START,
            task.END,
            task.TASK_TYPE,
            task_df,
            multitaskers_set,
            all_task_type_pairs
        )

    df_filtered['IS_FULLY_AVAILABLE'] = df_filtered.apply(is_fully_available, axis=1)
    df_filtered = df_filtered[df_filtered['IS_FULLY_AVAILABLE']].drop(columns=['IS_FULLY_AVAILABLE'])

    return df_filtered


In [16]:
"""
At this stage, we have a filtered dataframe with the translators that are available for the task,
it contains the following columns:
    - TRANSLATOR: Name of the translator
    - SOURCE_LANG: Source language of the translator
    - TARGET_LANG: Target language of the translator
    - HOURLY_RATE: Hourly rate of the translator
    - Filtered by the availability

    Things to take into account for the calculation of the scores:
    - AVG_QUALITY_BY_LNG: Average quality by language pair (if applicable)
    - AVG_QUALITY_BY_TASK: Average quality by task type (if applicable)
    - QUALITY_SOURCE_TASK: Source of the quality by task score (original, overall_penalized, global_penalized)
    - NUM_TASKS: Number of tasks performed by the translator (to take into account the reliability of the translator's quality and delay percentage, not experience because it is calculated based on specific tasks)

    - EXPERIENCE_SCORE: Experience score based on task type, language pair, industry, and subindustry
    - EXPERIENCE_CLIENT: Experience score based on the specific client (if applicable)
    - AVG_DELAY_PERCENTAGE: Average delay percentage of the translator (if applicable)

    
There are some key considerations regarding the experience and quality weights:
    - Proofreading and Spotcheck need more expereinced translators.
    - LanguageLead is a more advanced task, so it needs more experience and quality.
    - Test should be assigned to the most experienced and high-quality TRANSLATOR for the client or topic, regardless of price.
    - For training we dont need to take into account the experience nor the quality

"""

"\nAt this stage, we have a filtered dataframe with the translators that are available for the task,\nit contains the following columns:\n    - TRANSLATOR: Name of the translator\n    - SOURCE_LANG: Source language of the translator\n    - TARGET_LANG: Target language of the translator\n    - HOURLY_RATE: Hourly rate of the translator\n    - Filtered by the availability\n\n    Things to take into account for the calculation of the scores:\n    - AVG_QUALITY_BY_LNG: Average quality by language pair (if applicable)\n    - AVG_QUALITY_BY_TASK: Average quality by task type (if applicable)\n    - QUALITY_SOURCE_TASK: Source of the quality by task score (original, overall_penalized, global_penalized)\n    - NUM_TASKS: Number of tasks performed by the translator (to take into account the reliability of the translator's quality and delay percentage, not experience because it is calculated based on specific tasks)\n\n    - EXPERIENCE_SCORE: Experience score based on task type, language pair, in

In [ ]:

# ----- KNN ----
def knn(df_filtered, task, metric='euclidean', need_wildcard=False):
    """
    Optimized KNN to find the best translators based on the task's requirements,
    filtering out those unavailable based on multitasking constraints.
    """
    from sklearn.metrics import pairwise_distances

    # Prepare task vector
    task_vector = np.array([
        task.SELLING_HOURLY_PRICE,
        task.MIN_QUALITY,
        task.FORECAST
    ]).reshape(1, -1)

    # Prepare translator feature matrix
    translator_vectors = df_filtered[['HOURLY_RATE', 'QUALITY', 'NUM_TASKS']].values

    # Compute distances
    distances = pairwise_distances(translator_vectors, task_vector, metric=metric).flatten()

    # Filter by multitasking-aware availability
    available_mask = df_filtered.apply(
        lambda row: is_translator_available(
            row['TRANSLATOR'],
            task.START,
            task.END,
            task.TASK_TYPE,
            task_df,
            multitaskers_set,
            all_task_type_pairs
        ),
        axis=1
    )

    df_available = df_filtered[available_mask].copy()
    distances = distances[available_mask.values]

    # Sort and return top-k indices
    indexes = np.argsort(distances)
    return distances[indexes], df_available.iloc[indexes]


In [18]:
def evaluate_metrics(df_filtered, task):
    metrics = ['euclidean', 'manhattan']
    results = {}

    for metric in metrics:
        distances, indexes = knn(df_filtered, task, metric=metric, need_wildcard=False)
        selected_translators = get_best_translators(df_filtered, indexes, distances)
        
        # Guarda los resultados para cada métrica
        results[metric] = selected_translators
    
    return results


def plot_similarity(results):
    """
    Plot the similarity scores for translators selected using different distance metrics.
    
    Args:
        results (dict): A dictionary where keys are metric names and values are DataFrames 
                        with the selected translators' similarity scores.
    """
    # Create a DataFrame to hold all similarity scores
    similarity_data = []
    for metric, translators in results.items():
        # Add a new column with the metric name
        translators['Metric'] = metric
        similarity_data.append(translators)

    # Concatenate all DataFrames
    all_similarity_scores = pd.concat(similarity_data)

    # Plot using seaborn
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Metric', y='Similarity Score', data=all_similarity_scores)
    plt.title('Comparison of Similarity Scores for Different Metrics')
    plt.xlabel('Distance Metric')
    plt.ylabel('Similarity Score')
    plt.show()

Cosine similaruty is not considered, as it measures how similarly two vectors point in the same direction, regardless of their magnitude, and is used when relative proportions matter more than absolute values.

In [ ]:
# ----- MAIN CODE ----
# Creates a dataframe with  the additional attributes
translators_attributes_df = compute_delay_percentage(train_df, transl_cost_pairs_df)
translators_attributes_df = compute_number_tasks(train_df, translators_attributes_df) 

# Define the top-k value for evaluation
k = 10
correct_predictions = 0
total_predictions = 0

# Iterate over each row in the validation set
for idx, task_row in validation_df_clean.iterrows():
    #Convert the current row to a task
    new_task = task_row.copy()
    new_task = new_task.rename({'HOURLY_RATE': 'SELLING_HOURLY_PRICE', 'QUALITY_EVALUATION': 'MIN_QUALITY'})
    
    match = clients_df[clients_df['CLIENT_NAME'].str.strip() == new_task['MANUFACTURER'].strip()]

    if not match.empty:
        new_task['WILDCARD'] = match.iloc[0]['WILDCARD']
        new_task['HOURLY_RATE'] = match.iloc[0]['SELLING_HOURLY_PRICE']
        new_task['QUALITY_EVALUATION'] = match.iloc[0]['MIN_QUALITY']
    else:
        print("WARNING: No match found in schedules_df for the given client. Setting default values.")
        new_task['WILDCARD'] = 'Quality'
        new_task['HOURLY_RATE'] = new_task['SELLING_HOURLY_PRICE']
        new_task['QUALITY_EVALUATION'] = new_task['MIN_QUALITY']
    
    new_task = Task(**new_task.to_dict())  # Convert the task to the Task object
    
    # Filter translators based on task attributes
    df_filtered = filter_language_price_quality_availability(data_df, schedules_df, translators_attributes_df, new_task)
    
    if df_filtered.empty:
        print(f"\nNo available translators for task idx = {task_row['TASK_ID']}")
        print("\nTrying with the wildcard...\n")        
        df_filtered = filter_language_price_quality_availability(data_df, schedules_df, translators_attributes_df, new_task, need_wildcard = True)
        if df_filtered.empty:
            print("No available translators even with the wildcard")
            continue

    # Compute experience scores for the filtered translators
    compute_experience(df_filtered, task_type=new_task.TASK_TYPE, source_lang=new_task.SOURCE_LANG, target_lang=new_task.TARGET_LANG, industry=new_task.MANUFACTURER_INDUSTRY, subindustry=new_task.MANUFACTURER_SUBINDUSTRY)
    compute_experience_for_client(df_filtered, client=new_task.MANUFACTURER)

    # Get the distances and indexes from KNN
    distances, indexes = knn(df_filtered, new_task)
    
    # Get the best translators
    selected_translators = get_best_translators(df_filtered, indexes, distances)
    
    # Retrieve the true translator label for the current task
    true_translator = validation_translator_labels[idx]  # Assuming you have the true labels in the dictionary from earlier
    
    # Check if any of the top-k translators match the true translator
    top_k_translators = selected_translators.iloc[:k]  # Top-k translators
    if true_translator in top_k_translators['TRANSLATOR'].values:
        correct_predictions += 1
    
    total_predictions += 1

# Calculate the top-k accuracy
top_k_accuracy = correct_predictions / total_predictions
print(f"Top-{k} Accuracy: {top_k_accuracy * 100:.2f}%")


No available translators for task idx = 10431986. Possible reasons:
1. No translators available because the quality is too high
2. No translators available because the price is too low
3. No translators available because they are not available

Trying with the wildcard...

Empty DataFrame
Columns: [TRANSLATOR, SOURCE_LANG, TARGET_LANG, HOURLY_RATE, AVG_DELAY_PERCENTAGE, NUM_TASKS, AVG_QUALITY_BY_LG, AVG_QUALITY_BY_TASK, QUALITY_SOURCE_TASK, START, END, MON, TUE, WED, THU, FRI, SAT, SUN, EXPERIENCE_SCORE, EXPERIENCE_CLIENT]
Index: []
Empty DataFrame
Columns: [TRANSLATOR, SOURCE_LANG, TARGET_LANG, HOURLY_RATE, AVG_DELAY_PERCENTAGE, NUM_TASKS, AVG_QUALITY_BY_TASK, QUALITY_SOURCE_TASK, AVG_QUALITY_BY_LG, START, END, MON, TUE, WED, THU, FRI, SAT, SUN, EXPERIENCE_SCORE, EXPERIENCE_CLIENT]
Index: []

No available translators for task idx = 10417207. Possible reasons:
1. No translators available because the quality is too high
2. No translators available because the price is too low
3. No tra

ValueError: Input X contains NaN.
NearestNeighbors does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
new_task = validation_df_clean.loc[validation_df_clean['TASK_ID'] == 10189682].iloc[0]
# Convert the current row to a task
new_task = task_row.copy()
new_task = new_task.rename({'HOURLY_RATE': 'SELLING_HOURLY_PRICE', 'QUALITY_EVALUATION': 'MIN_QUALITY'})

print(new_task)

PROJECT_ID                                       219619
PM                                                  PMT
TASK_ID                                        11055080
START                               2021-09-23 11:50:00
END                                 2021-09-23 17:00:00
TASK_TYPE                                   Translation
SOURCE_LANG                                     English
TARGET_LANG                                      Basque
ASSIGNED                            2021-09-23 11:52:12
READY                               2021-09-23 11:52:19
WORKING                             2021-09-23 14:10:52
DELIVERED                           2021-09-23 16:58:51
RECEIVED                            2021-09-23 16:59:37
CLOSE                               2021-09-23 17:00:36
FORECAST                                           0.18
SELLING_HOURLY_PRICE                                 33
COST                                               5.94
MIN_QUALITY                                     

### Types of mistakes in the dataset they have been mostly added to the analysis to keep things clean

one of them is that some of the translators (13) finish after 0am 

In [ ]:
# hay algunos con fecha de START == END y si START > ASSIGNED (son errores porque no tiene sentido); pero si START <= ASSIGNED, 
# probablemente signifique que END (deadline) era urgente y no puso fecha límite, no son errores


# provisionalmente cambiamos el START como ASSIGNED para estos casos
# data_df.loc[data_df['START'] > data_df['ASSIGNED'], 'START'] = data_df['ASSIGNED'] 

## TODO we have to discuss this further, it have been added to data analysis but not shure how to address it 

### Next steps
actualize the unavailable translators:
- add it to the list when one is selected by the client in the forntend
- remove it from the list when the task has been finished 